### Connecting to APIs

Importing necessary packages

In [1]:
import requests
import pandas as pd
from configparser import ConfigParser

Reading API Key from configuration file, and setting connection parameters

In [48]:
config = ConfigParser()
config.read('config.ini')
token = config.get('tokens','api_key')
base_url = 'https://www.alphavantage.co/query?'
params = {
    "function": "TIME_SERIES_DAILY",
    "symbol": "AAPL",
    "apikey": f"{token}"}

In [49]:
#Making request to API endpoint
response = requests.get(base_url, params=params)
#Make sure we got a succesfull response
print(response)

<Response [200]>


In [50]:
#Converting response to json format
data = response.json()

Adding function that will parse and format the json response, and then convert it to pandas DataFrame

In [53]:
def df_from_response(data,
         columns=['Open', 'High', 'Low', 'Close', 'Volume']):

    def convert_response(d):
        # convert the response into datetimerecords that can be
        # parsed by Pandas
        for dt, prec in d['Time Series (Daily)'].items():
            r = { 'datetime': dt}
            r.update(prec)
            yield r

    # pass your response 'page'
    df = pd.DataFrame(convert_response(data))
    # rename the columns    
    df = df.rename(columns={'1. open': 'Open',
                            '2. high': 'High',
                            '3. low': 'Low',
                            '4. close': 'Close',
                            '5. volume': 'Volume'})
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    df.sort_index(inplace=True)
    # extract the default columns
    df = df[columns]
    return df

Applying function to the json data

In [54]:
df = df_from_response(data)
df

,Open,High,Low,Close,Volume
datetime,,,,,
2021-12-16,179.2800,181.1400,170.7500,172.2600,150185843
2021-12-17,169.9300,173.4700,169.6900,171.1400,195923441
2021-12-20,168.2800,170.5800,167.4600,169.7500,107499114
2021-12-21,171.5550,173.2000,169.1200,172.9900,91185905
2021-12-22,173.0400,175.8600,172.1500,175.6400,92135303
...,...,...,...,...,...
2022-05-04,159.6700,166.4800,159.2600,166.0200,107521649
2022-05-05,163.8500,164.0800,154.9500,156.7700,130525275
2022-05-06,156.0100,159.4400,154.1800,157.2800,116124647
